In [1]:
using Statistics, CSV, Tables, LinearAlgebra, SparseArrays

In [2]:
include("FVM.jl")

Main.FVM

In [3]:
include("solvers.jl")

Main.Solvers

## Función para construir la matriz a resolver

In [4]:
function Γ_constant(x::Array, y::Array, z::Array)
    Γ = 1000
    tensor_Γ = ones(length(x), length(y), length(z))
    tensor_Γ = Γ .* tensor_Γ
end

Γ_constant (generic function with 1 method)

In [5]:
function getsystem(vol)
    # Construcción de la malla.
    volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces = FVM.uniform_grid(vol, vol/10)
    tags = FVM.init_tags(3, volumes, centers_and_boundaries)
    tags_b = FVM.init_tags_boundaries(3, centers_and_boundaries)
    FVM.tag_wall(tags, tags_b, [:W, :E, :T, :N, :B], 0, :D)
    FVM.tag_wall(tags, tags_b, :S, 100, :D)
    mesh = FVM.Mesh(volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces, tags, tags_b)
    # Generando los coeficientes
    coeff = FVM.init_coefficients(mesh)
    FVM.set_diffusion(coeff, Γ_constant);
    # Construyendo el sistema de ecuaciones
    equation_system = FVM.init_eq_system(coeff)
    A = equation_system.A
    b = equation_system.b
    return A, b
end

getsystem (generic function with 1 method)

In [14]:
A, b = getsystem(30)

(sparse([1, 31, 1, 2, 32, 902, 2, 3, 33, 903  …  26098, 26968, 26998, 26999, 26099, 26969, 26999, 27000, 26970, 27000], [1, 1, 2, 2, 2, 2, 3, 3, 3, 3  …  26998, 26998, 26998, 26998, 26999, 26999, 26999, 26999, 27000, 27000], [1200.0000000000002, -100.00000000000001, -133.33333333333337, 1000.0000000000002, -100.00000000000001, -100.00000000000001, -133.33333333333337, 999.9999999999999, -99.99999999999997, -100.00000000000004  …  -100.00000000000054, -99.99999999999876, 999.9999999999943, -133.33333333333394, -99.99999999999964, -99.99999999999964, 999.9999999999997, -133.33333333333275, -99.99999999999876, 1199.9999999999925], 27000, 27000), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  26666.66666666648, 26666.666666666715, 26666.66666666648, 26666.666666666715, 26666.66666666648, 26666.66666666648, 26666.666666666715, 26666.66666666648, 26666.666666666715, 26666.66666666648])

In [21]:
D = Diagonal(A)

27000×27000 Diagonal{Float64, SparseVector{Float64, Int64}}:
 1200.0      ⋅       ⋅       ⋅       ⋅   …      ⋅       ⋅       ⋅       ⋅ 
     ⋅   1000.0      ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅   1000.0      ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅   1000.0      ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅   1000.0         ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅   …      ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅   …      ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅          ⋅       ⋅       ⋅       ⋅ 
     ⋅       ⋅       ⋅       ⋅       ⋅ 

In [22]:
D⁻¹ = inv(D)

27000×27000 Diagonal{Float64, SparseVector{Float64, Int64}}:
 0.000833333   ⋅      ⋅      ⋅      ⋅     …   ⋅      ⋅      ⋅      ⋅ 
  ⋅           0.001   ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅     0.001   ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅     0.001   ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅     0.001      ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅     …   ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅     …   ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
  ⋅            ⋅      ⋅      ⋅      ⋅         ⋅      ⋅      ⋅      ⋅ 
 ⋮                           

## Benchmarking

In [41]:
function iteratevolumes(volumes::Array{Int, 1}, times::Int)
    for volume ∈ volumes
        A, b = getsystem(volume)
        solvers = [Solvers.vectorizedjacobi, 
        #Solvers.paralleljacobi, 
        Solvers.vectorizedgaussseidel, 
        #Solvers.parallelgaussseidel]
        Solvers.vectorizedsor, 
        #Solvers.parallelsor, 
        Solvers.gmres,
        #Solvers.parallelgmres, 
        Solvers.gmresreiniciado, 
        #Solvers.parallelgmresreiniciado, 
        Solvers.gmresprecondicionado, 
        #Solvers.parallelgmresprecondicionado,
        Solvers.gmresprecondicionadoreiniciado]
        #Solvers.parallelgmresprecondicionadoreiniciado]
        names = ["Jacobi", "Gauss-Seidel", "SOR"]
        for solver ∈ solvers
            println("Comencé el de $volume volúmenes con el solver $solver")
            if occursin("precondicionado",string(solver))
                println("Precondicionado")
                for name ∈ names
                    list_of_time_statistics = list_of_statistics(solver, name, A, b, volume, times)
                    writetofile(solver, name, list_of_time_statistics')
                end
            else
                list_of_time_statistics = list_of_statistics(solver, A, b, volume, times)
                writetofile(solver, list_of_time_statistics')
            end
        end
        println("Terminé el de $volume volúmenes")
    end
end

iteratevolumes (generic function with 1 method)

In [26]:
function list_of_statistics(solver, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, volume::Int, times::Int)
    time_list, μ, σ, iterations_list, μ_iterations, σ_iterations = getstatistics(solver, A, b, times)
    time_statistics = [volume, volume^3, μ, σ, μ_iterations, σ_iterations]
    push!(time_statistics, time_list...)
    push!(time_statistics, iterations_list...)
    return time_statistics
end

list_of_statistics (generic function with 1 method)

In [27]:
function list_of_statistics(solver, name::String, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, volume::Int, times::Int)
    time_list, μ, σ, iterations_list, μ_iterations, σ_iterations = getstatistics(solver, name, A, b, times)
    time_statistics = [volume, volume^3, μ, σ, μ_iterations, σ_iterations]
    push!(time_statistics, time_list...)
    push!(time_statistics, iterations_list...)
    return time_statistics
end

list_of_statistics (generic function with 2 methods)

In [28]:
function getstatistics(solver, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, times::Int)
    time_list = []
    iterations_list = []
    for i ∈ 1:(times+1)
        t, iterations = measuretime(solver, A, b)
        push!(time_list, t)
        push!(iterations_list, iterations)
    end
    times_without_compiling = time_list[2:end]
    iterations_without_compiling = iterations_list[2:end]
    μ = mean(times_without_compiling)
    σ = std(times_without_compiling)
    μ_iterations = mean(iterations_without_compiling)
    σ_iterations = std(iterations_without_compiling)
    
    return time_list, μ, σ, iterations_without_compiling, μ_iterations, σ_iterations
end

getstatistics (generic function with 1 method)

In [29]:
function getstatistics(solver, name::String, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, times::Int)
    time_list = []
    iterations_list = []
    for i ∈ 1:(times+1)
        t, iterations = measuretime(solver, name, A, b)
        push!(time_list, t)
        push!(iterations_list, iterations)
    end
    times_without_compiling = time_list[2:end]
    iterations_without_compiling = iterations_list[2:end]
    μ = mean(times_without_compiling)
    σ = std(times_without_compiling)
    μ_iterations = mean(iterations_without_compiling)
    σ_iterations = std(iterations_without_compiling)
    
    return time_list, μ, σ, iterations_without_compiling, μ_iterations, σ_iterations
end

getstatistics (generic function with 2 methods)

In [30]:
function measuretime(solver, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64})
    start_time = time()
    iterations = solver(A,b)
    finish_time = time()
    t = finish_time - start_time
    return t, iterations
end

measuretime (generic function with 1 method)

In [31]:
function measuretime(solver, name, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64})
    start_time = time()
    iterations = solver(A, b, name)
    finish_time = time()
    t = finish_time - start_time
    return t, iterations
end

measuretime (generic function with 2 methods)

In [32]:
function writetofile(solver, list)
    file_name = string(solver)
    CSV.write("../../benchmarking/solvers/$(file_name).csv", Tables.table(list), delim = ',',append=true)
    #CSV.write("benchmarking/$(file_name).csv", Tables.table(list), delim = ',',append=true)
end

writetofile (generic function with 2 methods)

In [33]:
function writetofile(solver, name, list)
    file_name = string(solver)
    CSV.write("../../benchmarking/solvers/$(file_name)_$(name).csv", Tables.table(list), delim = ',',append=true)
end

writetofile (generic function with 2 methods)

In [42]:
volumes = [10]
times = 10
iteratevolumes(volumes, times)

Comencé el de 10 volúmenes con el solver vectorizedjacobi
../../benchmarking/solvers/vectorizedjacobi.csv
Comencé el de 10 volúmenes con el solver vectorizedgaussseidel
../../benchmarking/solvers/vectorizedgaussseidel.csv
Comencé el de 10 volúmenes con el solver vectorizedsor
../../benchmarking/solvers/vectorizedsor.csv
Comencé el de 10 volúmenes con el solver gmres
../../benchmarking/solvers/gmres.csv
Comencé el de 10 volúmenes con el solver gmresreiniciado
../../benchmarking/solvers/gmresreiniciado.csv
Comencé el de 10 volúmenes con el solver gmresprecondicionado
Precondicionado
string(solver) = "gmresprecondicionado"
name = "Jacobi"
../../benchmarking/solvers/gmresprecondicionado_Jacobi.csv
string(solver) = "gmresprecondicionado"
name = "Gauss-Seidel"
../../benchmarking/solvers/gmresprecondicionado_Gauss-Seidel.csv
string(solver) = "gmresprecondicionado"
name = "SOR"
../../benchmarking/solvers/gmresprecondicionado_SOR.csv
Comencé el de 10 volúmenes con el solver gmresprecondicionado

---